In [1]:
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_experimental.agents.agent_toolkits.pandas.base import _get_functions_single_prompt
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain_openai import ChatOpenAI
import pandas as pd

/Users/sahibpreetsingh/miniconda3/envs/sahib_lit/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Create a temporary DataFrame with sample data
data = {
    "Jersey Color": ["Red", "Blue", "Green", "Yellow", "Black"],
    "Team Name": ["Team A", "Team B", "Team C", "Team D", "Team E"],
    "Country Name": ["Country X", "Country Y", "Country Z", "Country W", "Country V"]
}

df = pd.DataFrame(data)

# Display the temporary DataFrame
print(df)

  Jersey Color Team Name Country Name
0          Red    Team A    Country X
1         Blue    Team B    Country Y
2        Green    Team C    Country Z
3       Yellow    Team D    Country W
4        Black    Team E    Country V


In [3]:
# This is a hack
prompt = _get_functions_single_prompt(df)
prompt.input_variables.append("chat_history")
prompt.messages.insert(1, MessagesPlaceholder(variable_name="chat_history"))

In [4]:


tools = [PythonAstREPLTool( locals={"df": df},description = """
A Python shell. Use this to execute python commands. Input should be a valid python command. 
Try and break the problem in steps and solve the prblem step by step.
""")]

chat_model = ChatOpenAI(model="gpt-4-0613")
agent = create_openai_functions_agent(chat_model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [5]:
chat_history = []
def memory_maker(query):
    chat_history.extend(
    [
        HumanMessage(content=query),
        AIMessage(content=response["output"])
    ],
    )
    return chat_history

In [6]:


query = "how many rows are in the dataframe?"

response = agent_executor.invoke({"input": query, "chat_history": chat_history})
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape[0]'}`


5The dataframe has 5 rows.

> Finished chain.
The dataframe has 5 rows.


In [7]:
query='which team has blue jersey'

In [8]:


# query = "What were we talking about?"
response = agent_executor.invoke({"input": query, "chat_history": memory_maker(query)})
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Jersey Color'] == 'Blue']['Team Name']"}`


1    Team B
Name: Team Name, dtype: objectThe team with a blue jersey is Team B.

> Finished chain.
The team with a blue jersey is Team B.


In [9]:
query="and what's the country name of it?"

In [10]:
response = agent_executor.invoke({"input": query, "chat_history": memory_maker(query)})
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.loc[df['Jersey Color'] == 'Blue', 'Country Name'].values[0]"}`


Country YThe country of the team with a blue jersey is Country Y.

> Finished chain.
The country of the team with a blue jersey is Country Y.
